In [63]:
from ews.models import *
import pandas as pd
#import geopandas
import geojson
import matplotlib.pyplot as plt
import numpy as np


In [64]:
Site.objects.all()

<QuerySet [<Site: neuilly_sur_marne_smv17>, <Site: chenneviere_si>, <Site: pont_d'iéna_rg>, <Site: pont_de_l'alma_rg>, <Site: marne_dwc_2019>, <Site: pont_alma_dwc_2019>, <Site: seine_valenton>, <Site: marne_aval_wwtp>, <Site: fresnes_choisy>, <Site: saint_baudile>, <Site: canal_chesney>, <Site: centre_urbain>, <Site: neuilly_gagny>, <Site: ru_de_nelse>, <Site: maltournée>, <Site: avenue_de_neptune_nogent>, <Site: alfortville_flow>, <Site: bartel_flow>, <Site: charenton_flow>, <Site: des_4_flow>, '...(remaining elements truncated)...']>

In [65]:
loc = Site()

In [66]:
d = pd.read_csv("D:/DWC_plattform/metadata_clean.csv")

In [67]:
d

,SiteName,SiteReferencementName,SiteType,SourceID,SiteLatitude(y),SiteLongitude(x)
0,neuilly_sur_marne_smv17,SMV17 - Neuilly-sur-Marne,River,SMV,48.852620,2.522907
1,chenneviere_si,Chenneviere,River,SIAAP,48.790690,2.521590
2,pont_d'iéna_rg,Pont d'Iéna (RG),River,VDP,48.859478,2.292624
3,pont_de_l'alma_rg,Pont de l'Alma (RG),River,VDP,48.863021,2.301786
4,marne_dwc_2019,Marne_DWC_2019,River,SIAAP,48.853460,2.530100
...,...,...,...,...,...,...
121,marine,DO27505634-DO Marine,Network,VDP,48.844970,2.367150
122,mirabeau_convention,DO27505639-DO Mirabeau-Convention,Network,VDP,48.846890,2.276920
123,javel_leblanc,DO27505640-DO Javel-Leblanc,Network,VDP,48.841620,2.271710
124,wilson,DO29204430-DO Wilson,Network,VDP,48.900820,2.282630


In [68]:
#FeatureType.objects.bulk_create([FeatureType(name="WWTP"), FeatureType(name="Network"), FeatureType(name="River")])

In [69]:
Site.objects.all().delete()

(126, {'ews.Site': 126})

In [70]:
FeatureType.objects.get(name = d.SiteType[0])

<FeatureType: River>

In [71]:
for index, row in d.iterrows():
    loc = Site()
    loc.name = row['SiteName']
    loc.ref_name = row['SiteReferencementName']
    loc.geom = {"coordinates": [row['SiteLongitude(x)'],row['SiteLatitude(y)']], "type": "Point"}
    loc.feature_type = FeatureType.objects.get(name = row['SiteType'])
    loc.owner = User.objects.get(id = 1)
    loc.save()

In [72]:
flows = pd.read_csv('D:/DWC_plattform/flows.csv',parse_dates = True, index_col = "date")

In [73]:
flows.head()

,value,site
date,,
2018-12-31 23:00:00+00:00,0.0,avenue_de_neptune_nogent
2018-12-31 23:05:00+00:00,0.0,avenue_de_neptune_nogent
2018-12-31 23:10:00+00:00,0.0,avenue_de_neptune_nogent
2018-12-31 23:15:00+00:00,0.0,avenue_de_neptune_nogent
2018-12-31 23:20:00+00:00,0.0,avenue_de_neptune_nogent


In [74]:
sites = flows.site.unique()

In [75]:
select = flows[flows["site"]==sites[0]]

In [76]:
daily = select.resample("6H").mean()

In [77]:
aggregates =flows.groupby("site").resample("6H").mean()

In [78]:
d = aggregates.reset_index()

In [79]:
d.value = d.value.round(3)

In [80]:
d.sort_values(by = "value")

,site,date,value
114268,canal_chesney,2015-07-05 00:00:00+00:00,-9999.0
114270,canal_chesney,2015-07-05 12:00:00+00:00,-9999.0
114267,canal_chesney,2015-07-04 18:00:00+00:00,-9999.0
114266,canal_chesney,2015-07-04 12:00:00+00:00,-9999.0
114265,canal_chesney,2015-07-04 06:00:00+00:00,-9999.0
...,...,...,...
133076,cd52,2019-05-21 06:00:00+00:00,NaN
133077,cd52,2019-05-21 12:00:00+00:00,NaN
133078,cd52,2019-05-21 18:00:00+00:00,NaN
133079,cd52,2019-05-22 00:00:00+00:00,NaN


In [81]:
d.loc[d["value"] < 0, ["value"]] = np.nan

In [82]:
d = d.dropna()

In [83]:
#FeatureData.objects.all().delete()

(0, {})

In [84]:
y=d.sort_values(by = "value", ascending = False).tail(3)

In [85]:
y

,site,date,value
48122,avenir,2019-10-30 18:00:00+00:00,0.0
48121,avenir,2019-10-30 12:00:00+00:00,0.0
133974,cd52,2019-12-31 18:00:00+00:00,0.0


In [86]:
list = []
for index, row in d.iterrows():
    dpoint = FeatureData()
    dpoint.date = row["date"]
    dpoint.value = row["value"]
    dpoint.site = Site.objects.get(name = row['site'])
    dpoint.save()
    
    
